In [2]:
# Import necessary libraries
import pymongo
from pymongo import MongoClient
import joblib
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from flask import Flask, jsonify, request
import jsonpickle

# Connect to MongoDB
app = Flask(__name__)
client = MongoClient('mongodb://localhost:27017/')

# Access the database
db = client['Pharmacy']

# Access the collection
collection = db['sales']

# Load the Jupyter Lab prediction model
@app.route('/api/v1/model', methods=['GET'])
def get_model():
    model = joblib.load('./pharma_model.pkl')
    return model

model=get_model()

def predict_sales(data): 
    # Generate a range of dates
    # print(data)

    dates = pd.date_range(start=data['startDate'], end=data['endDate'], freq='D')
    # Create the DataFrame with dates as the index
    df_test = pd.DataFrame(index=dates)    
    df_test['Year'] = df_test.index.year
    df_test['Month'] = df_test.index.month
    df_test['Weekday Name'] = df_test.index.weekday
    df_test['day'] = df_test.index.day
    df_test['Drug'] = data['drug']
    print(data)
    
    le = LabelEncoder()
    # df_test['Weekday Name'] = le.fit_transform(df_test['Weekday Name'])
    
    df_test['predicted_quantity'] = model.predict(df_test)
    df_test.index = df_test.index.astype(str)

    # Convert the DataFrame to a dictionary
    result_dict = df_test.to_dict()
    
    return result_dict

# Store the predictions in MongoDB
#collection.insert_many(predictions)


In [3]:
@app.route('/api/v1/predict', methods=['POST'])
def predict():
    data = request.get_json()

    # Assuming you have a model function for prediction
    result = predict_sales(data)
    # result_dict=result.to_dict()
    # result = list(enumerate(result_dict))
    return jsonify(result)
    # jsonEncoded=jsonpickle.encode(result)
    # print(result)
    # return jsonEncoded
    # return jsonify({'prediction': result_dict})




In [4]:
if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Dec/2023 12:32:54] "POST /api/v1/predict HTTP/1.1" 200 -


{'drug': 5, 'startDate': '2023-12-31', 'endDate': '2023-12-31'}


127.0.0.1 - - [01/Dec/2023 12:33:31] "POST /api/v1/predict HTTP/1.1" 200 -


{'drug': 5, 'startDate': '2023-12-01', 'endDate': '2023-12-31'}


127.0.0.1 - - [01/Dec/2023 12:34:15] "POST /api/v1/predict HTTP/1.1" 200 -


{'drug': 1, 'startDate': '2023-12-01', 'endDate': '2023-12-31'}
